# Clean-up Vector Search (formerly Matching Engine) Indexes and Index Endpoints

In [13]:
# naming convention for all cloud resources
VERSION        = "v1"                  # TODO
PREFIX         = f'ndr-{VERSION}'      # TODO

print(f"PREFIX = {PREFIX}")

PREFIX = ndr-v1


In [14]:
# staging GCS
GCP_PROJECTS             = !gcloud config get-value project
PROJECT_ID               = GCP_PROJECTS[0]

# GCS bucket and paths
BUCKET_NAME              = f'{PREFIX}-{PROJECT_ID}-bucket'
BUCKET_URI               = f'gs://{BUCKET_NAME}'

config = !gsutil cat {BUCKET_URI}/config/notebook_env.py
print(config.n)
exec(config.n)


PROJECT_ID               = "hybrid-vertex"
PROJECT_NUM              = "934903580331"
LOCATION                 = "us-central1"

REGION                   = "us-central1"
BQ_LOCATION              = "US"
VPC_NETWORK_NAME         = "ucaip-haystack-vpc-network"

VERTEX_SA                = "934903580331-compute@developer.gserviceaccount.com"

PREFIX                   = "ndr-v1"
VERSION                  = "v1"

APP                      = "sp"
MODEL_TYPE               = "2tower"
FRAMEWORK                = "tfrs"
DATA_VERSION             = "v1"
TRACK_HISTORY            = "5"

BUCKET_NAME              = "ndr-v1-hybrid-vertex-bucket"
BUCKET_URI               = "gs://ndr-v1-hybrid-vertex-bucket"
SOURCE_BUCKET            = "spotify-million-playlist-dataset"

DATA_GCS_PREFIX          = "data"
DATA_PATH                = "gs://ndr-v1-hybrid-vertex-bucket/data"
VOCAB_SUBDIR             = "vocabs"
VOCAB_FILENAME           = "vocab_dict.pkl"

CANDIDATE_PREFIX         = "candidates"
TRAIN_DIR_PREFIX      

In [15]:
import os
import sys
import time
import numpy as np

# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

# google cloud SDKs
from google.cloud import storage
from google.cloud import aiplatform as vertex_ai

In [16]:
vertex_ai.init(project=PROJECT_ID, location=LOCATION)

storage_client = storage.Client(project=PROJECT_ID)

## Index Endpoints

In [42]:
!gcloud beta ai index-endpoints list --project=$PROJECT_ID --region=us-central1

Using endpoint [https://us-central1-aiplatform.googleapis.com/]
---
createTime: '2023-09-25T18:17:22.236805Z'
description: endpoint for ann index
displayName: ann_index_endpoint_v1
encryptionSpec: {}
etag: AMEw9yOtL9anSeZQNxojWASyBixy13kG2ZWcexe13udTtSsCl6k6XRSfcBGWkDhJw6UT
name: projects/934903580331/locations/us-central1/indexEndpoints/7571386602446913536
network: projects/934903580331/global/networks/ucaip-haystack-vpc-network
updateTime: '2023-09-25T18:17:22.946019Z'
---
createTime: '2023-09-25T17:53:18.383235Z'
deployedIndexes:
- automaticResources:
    maxReplicaCount: 2
    minReplicaCount: 2
  createTime: '2023-09-25T18:29:41.235899Z'
  deploymentGroup: default
  id: deployedbf_v1
  index: projects/934903580331/locations/us-central1/indexes/1095773288241561600
  indexSyncTime: '2023-09-25T18:32:44.848032Z'
  privateEndpoints:
    matchGrpcAddress: 10.41.2.5
description: endpoint for brute force index
displayName: bf_index_endpoint_v1
encryptionSpec: {}
etag: AMEw9yMCvj1fMxOMel9

In [36]:
TO_DELETE = "projects/934903580331/locations/us-central1/indexEndpoints/1049611392061014016" # XXX

my_index_endpoint = vertex_ai.MatchingEngineIndexEndpoint(index_endpoint_name=TO_DELETE)

my_index_endpoint

resource name: projects/934903580331/locations/us-central1/indexEndpoints/1049611392061014016

In [37]:
my_index_endpoint.deployed_indexes

[]

In [38]:
my_index_endpoint.undeploy_all()

resource name: projects/934903580331/locations/us-central1/indexEndpoints/1049611392061014016

In [39]:
my_index_endpoint.delete(force=True)

Deleting MatchingEngineIndexEndpoint : projects/934903580331/locations/us-central1/indexEndpoints/1049611392061014016
Delete MatchingEngineIndexEndpoint  backing LRO: projects/934903580331/locations/us-central1/operations/7749232264642297856
MatchingEngineIndexEndpoint deleted. . Resource name: projects/934903580331/locations/us-central1/indexEndpoints/1049611392061014016


## Vector Search (Matching Engine) Indexes

In [6]:
# !gcloud beta ai index-endpoints list --project=$PROJECT_ID --region=us-central1

In [25]:
INDEX_TO_DELETE = "projects/934903580331/locations/us-central1/indexes/6250846749208870912" # 6283920058972372992

my_index = vertex_ai.MatchingEngineIndex(index_name=INDEX_TO_DELETE)

my_index

resource name: projects/934903580331/locations/us-central1/indexes/6250846749208870912

In [26]:
# Delete indexes
my_index.delete()

Deleting MatchingEngineIndex : projects/934903580331/locations/us-central1/indexes/6250846749208870912
Delete MatchingEngineIndex  backing LRO: projects/934903580331/locations/us-central1/indexes/6250846749208870912/operations/2732785229704986624
MatchingEngineIndex deleted. . Resource name: projects/934903580331/locations/us-central1/indexes/6250846749208870912


In [41]:
!gcloud beta ai index-endpoints list --project=$PROJECT_ID --region=us-central1

Using endpoint [https://us-central1-aiplatform.googleapis.com/]
---
createTime: '2023-09-25T18:17:22.236805Z'
description: endpoint for ann index
displayName: ann_index_endpoint_v1
encryptionSpec: {}
etag: AMEw9yNCsIxFg47Orv4AU4D2CbGrrPZNiNgGcn60WEwbX5iaB-QI4fg4kEGfzfkyuweV
name: projects/934903580331/locations/us-central1/indexEndpoints/7571386602446913536
network: projects/934903580331/global/networks/ucaip-haystack-vpc-network
updateTime: '2023-09-25T18:17:22.946019Z'
---
createTime: '2023-09-25T17:53:18.383235Z'
description: endpoint for brute force index
displayName: bf_index_endpoint_v1
encryptionSpec: {}
etag: AMEw9yMsnxqyX37gZCDgD-u4FqDlnA6T_-J9FsBfjhDNyanNpKB8tJ20X_36wAmvoFL4
name: projects/934903580331/locations/us-central1/indexEndpoints/7850046829390462976
network: projects/934903580331/global/networks/ucaip-haystack-vpc-network
updateTime: '2023-09-25T17:53:18.968009Z'
---
createTime: '2023-09-19T16:57:46.998462Z'
deployedIndexes:
- automaticResources:
    maxReplicaCount: